# Exploratory Data Analysis

The steps for the process EDA is:

## 1. Load the file of IDEAM stations in Colombia and filter by the department of Boyacá

In [1]:
#Start by connecting gdrive into the google colab
from google.colab import drive
drive.mount('/content/gdrive') 

ModuleNotFoundError: No module named 'google'

In [6]:
!ls "/content/gdrive/MyDrive/phd/deployment proyect/Data/IDEAM_2022"

 BoyacaPrecipitation.rar
 CNE_IDEAM.xlsx
'Configuración Regional de Números en Windows.pdf'
 GlosarioVariables.xlsx
'Instructivo Lectura de archivos .Data.pdf'
 RAD_20229050172462
 RAD_20229050172462.zip


In [7]:
import pandas as pd
# Read Excel file
df = pd.read_excel('/content/gdrive/MyDrive/phd/deployment proyect/Data/IDEAM_2022/CNE_IDEAM.xlsx')
df_boyaca = df[df['DEPARTAMENTO'] == 'Boyacá']

## 2. Search for files corresponding to daily weather variables
Some variables of importance to the subject of the study are:
- evaporation: EV_TT_D: Total daily evaporation
- sun: BSHG_TT_D: Total daily sunshine
- temmax: THSM_MX_D: daily Maximum humid temperature
- temmin: THSM_MN_D: Daily minimum humid temperature
- humidity: HRHG_MEDIA_D: Average daily relative humidity
- speed-wind: VVAG_MEDIA_D: Average daily wind speed
- precipitation: PT_10_TT_D: Total daily precipitation

In [9]:
import os
nombre_carpeta = '/content/gdrive/MyDrive/phd/deployment proyect/Data/IDEAM_2022/RAD_20229050172462'
contenido = os.listdir(nombre_carpeta)
files = []
for elemento in contenido:
    if elemento != 'desktop.ini' and 'EV_TT_D' in elemento:
        files.append(elemento)
    if elemento != 'desktop.ini' and 'BSHG_TT_D' in elemento:
        files.append(elemento)
    if elemento != 'desktop.ini' and 'THSM_MX_D' in elemento:
        files.append(elemento)
    if elemento != 'desktop.ini' and 'THSM_MN_D' in elemento:
        files.append(elemento)
    if elemento != 'desktop.ini' and 'HRHG_MEDIA_D' in elemento:
        files.append(elemento)
    if elemento != 'desktop.ini' and 'VVAG_MEDIA_D' in elemento:
        files.append(elemento)
    if elemento != 'desktop.ini' and 'PT_10_TT_D' in elemento:
        files.append(elemento)

## 3. Creation of a dataframe with station data

Separate data from file names: TYPE_SENSOR@STATION.data as follows:

| sensor | station | date | value |

the original date format is yyyy-mm-dd hh:mm:ss as it is total cumulative it is not worth having time so the idea is to convert only to yyyy-mm-dd

In [10]:
data = []
list_dataframes = []
for f in files:
    sensor = f[0:str(f).index('@')]
    station = f[str(f).index('@')+1:-5]
    data = pd.read_fwf(os.path.join(nombre_carpeta, f))
    data['sensor'] = sensor
    data['station'] = station
    list_dataframes.append(data)

frame_station = pd.concat(list_dataframes, axis=0, ignore_index=True)
frame_station[['date','value']] = frame_station['Fecha|Valor'].str.split('|',expand=True)
frame_station_final = frame_station.drop(['Fecha|Valor'], axis=1)
frame_station_final['date'] = frame_station_final['date'].str[:10]
frame_station_final

,sensor,station,date,value
0,PT_10_TT_D,35215030,2017-08-19,0.0
1,PT_10_TT_D,35215030,2017-08-20,35.79999999895
2,PT_10_TT_D,35215030,2017-08-21,8.1
3,PT_10_TT_D,35215030,2017-08-22,6.6000000000000005
4,PT_10_TT_D,35215030,2017-08-23,0.0
...,...,...,...,...
55144,PT_10_TT_D,24035360,2022-08-28,6.2
55145,PT_10_TT_D,24035360,2022-08-29,4.7
55146,PT_10_TT_D,24035360,2022-08-30,0.5
55147,PT_10_TT_D,24035360,2022-08-31,0.0


In [11]:
frame_station_final.describe()

,sensor,station,date,value
count,55149,55149,55149,55149
unique,1,21,6345,3389
top,PT_10_TT_D,24035360,2018-11-06,0.0
freq,55149,6345,21,27124


In [12]:
frame_station_final.date.describe()

count          55149
unique          6345
top       2018-11-06
freq              21
Name: date, dtype: object